In [ ]:
# 🛑 Import des bibliothèques
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy import signal, fft
import time

# À propos du théorème de Plancherel et de la complexité de calcul

Dans cet exercice, on se propose de vérifier expérimentalement une conséquence du théorème de Plancherel. Ce dernier, grâce à la relation $\mathcal{F}(x\ast y) = \mathcal{F}(x)\times \mathcal{F}(y)$, ou, autrement dit $(x\ast y) = \mathcal{F}^{-1}\big(\mathcal{F}(x)\times \mathcal{F}(y)\big)$, permet de réduire la complexité effective du calcul de $(x\ast y)$ de $\mathcal{O}(N^2)$ par la formule directe à $\mathcal{O}(N \log(N))$ pour des signaux de longueur $N$.

Ce tour de force réside dans le fait que la transformée de Fourier d'un signal de longueur $N$ peut se calculer en $\mathcal{O}(N \log(N))$ grâce à l'algorithme de <b>transformée de Fourier rapide</b> (*FFT = Fast Fourier Transform*), dont vous pouvez trouver plus de détails :
* Sur la [page Wikipédia](https://en.wikipedia.org/wiki/Fast_Fourier_transform) de la FFT.
* Dans cette vidéo d'[introduction générale à la FFT](https://www.youtube.com/watch?v=E8HeD-MUrjY&ab_channel=SteveBrunton) et dans cette vidéo d'[esquisse de l'algorithme FFT](https://www.youtube.com/watch?v=toj_IoCQE-4&ab_channel=SteveBrunton)
* Dans cette incroyable vidéo expliquant le [principe de la FFT comme un produit de polynômes](https://www.youtube.com/watch?v=h7apO7q16V0&ab_channel=Reducible)

À noter que le prochain TP portera exclusivement sur la réimplémentation de l'algorithme de transformée de Fourier rapide. Vous pouvez donc aussi considérer ce résultat comme acquis pour le moment, et patienter encore une semaine pour en avoir une preuve directe 😉

## Vérification pratique que la FFT est bien de complexité $\mathcal{O}(N \log(N))$

Bien qu'on puisse appliquer l'algorithme FFT sur des signaux de n'importe quelle taille, le cas le plus favorable (aussi bien du point de vue de son implémentation que de son explication algorithmique) est lorsque $N$ est une puissance de $2$ : $N = 2^n$. On va se restreindre à ce cas dans cet exercice.

Pour s'assurer de la complexité algorithmique en $\mathcal{O}(N \log(N))$ de la FFT, il faudrait en théorie monitorer le nombre d'opérations effectuées en interne lors d'un appel à `sp.fft.fft`. En pratique, c'est particulièrement complexe à réaliser, à moins de réécrire l'intégralité du code soi même (ce qui est évidemment hors de propos ici). Ainsi, on va plutôt monitorer le temps de calcul d'un appel à `sp.fft.fft` grâce à la routine `time.time()`. De base, cette dernière renvoie le temps actuel, mesuré en secondes depuis le 1er janvier 1970 à 00:00:00 UTC. En l'appelant une fois avant l'appel à `sp.fft.fft` et une autre fois juste après, puis en faisant la différence des deux, on obtient donc le temps écoulé lors de l'appel à `sp.fft.fft`.

In [ ]:
# 🛑 Utilisation de time.time()
t_now = time.time()
print(t_now)

In [ ]:
# 🛑 Monitorer le temps d'exécution d'une fonction / partie de code
t_begin = time.time()
for ii in range(10):
    time.sleep(0.2) # pause de 0.2 secondes
t_end = time.time()
t_elapse = t_end - t_begin
print("Temps écoulé : ",t_elapse)

Pour maintenant vérifier si le temps d'exécution de `sp.fft.fft` est bien en $\mathcal{O}(N \log(N))$ lorsque $N$ est une puissance de 2, on va avoir besoin de plusieurs choses :
* une liste de valeurs de la forme $2^n$ (lorsque $n$ varie entre $n_{min}$ et $n_{max}$) qui vont correspondre aux longueurs des signaux dont on va monitorer le temps de calcul de la FFT. En pratique, on va prendre $n_{min} = 2$ (donc la longueur minimale des signaux sera de $2^2 = 4$ points) et $n_{max} = 2^{22}$ (donc la longueur maximale des signaux sera de $2^{22} = 4 194 304 \simeq 4.2 \times 10^6$ points). Libre à vous de prendre plus pour $n_{max}$, mais attendez vous potentiellement à ce que ça mette un peu de temps pour tourner, en fonction du PC sur lequel vous êtes en train de travailler...

In [ ]:
# 🛑 liste de valeurs de la forme 2^n
list_N = 2**np.arange(2,23) # on prend toutes les puissances de 2 entre 2^2 et 2^22
print("Liste des valeurs de la forme 2^n :",list_N)

* une méthode de génération automatique de signaux, en fonction d'une taille $N$ donnée. On va essayer d'éviter ici des signaux "évidents" (par exemple signal constant, signal nul sauf en quelques points, etc). On définit donc ci-dessous 3 lambdas pour générer des signaux en fonction d'une taille donnée :
    * un signal dont les $N$ valeurs sont générées aléatoirement grâce à la lambda `generate_rand`
    * un signal en dent de scie, grâce à la lambda `generate_sawtooth`
    * un signal créneau, grâce à la lambda `generate_square`

In [ ]:
# 🛑 fonction génératrices de signaux de longueur N
generate_rand = lambda N : np.random.randn(N) # signal aléatoire
generate_sawtooth = lambda N : sp.signal.sawtooth(np.log2(N)*np.linspace(0,np.log2(N),N)) # signal dents de scie
generate_square = lambda N : sp.signal.square(np.log2(N)*np.linspace(0,np.log2(N),N)) # signal carré

In [ ]:
# 🛑 Tracé de ces différents signaux
Ntest = 2**8

plt.figure(figsize=(15,5))
plt.subplot(1,3,1)
plt.title('signal aléatoire')
plt.plot(generate_rand(Ntest),'b')
plt.xlim((0,Ntest-1))
plt.subplot(1,3,2)
plt.title('signal dent de scie')
plt.plot(generate_sawtooth(Ntest),'b')
plt.xlim((0,Ntest-1))
plt.subplot(1,3,3)
plt.title('signal carré')
plt.plot(generate_square(Ntest),'b')
plt.xlim((0,Ntest-1))
plt.show()

### 🛠️ 🚧 👷  À vous de jouer !

Complétez la cellule suivante pour calculer, pour chaque longueur $N$ dans `list_N`, le temps de calcul de la FFT d'un signal de longueur $N$ (pour une forme de signal donnée). Tracez ensuite le temps de calcul en fonction de $N$. <br>
L'évolution du temps de calcul de la FFT est elle dépendante de la forme du signal choisie ?
Que pouvez-vous en conclure quant au fait que l'algorithme FFT est bien en $\mathcal{O}(N \log(N))$ ?

In [ ]:
t_elapse = [] # initialisation de la liste pour stocker les temps de calcul

for N in list_N:
    x = generate_??? # FIXME génère un signal aléatoire/dent de scie/carré de longueur N
    t_begin = ??? # FIXME début chronomètre
    dummy_fft = ??? # FIXME calcul de la FFT du signal
    t_end = ??? # FIXME fin chronomètre
    t_elapse.append(t_end - t_begin) # insert temps de calcul dans la liste prévue à cet effet

In [ ]:
# 🛑 Temps de calcul vs O(Nlog(N))
plt.figure(figsize=(8,5))
plt.loglog(list_N,t_elapse,'b',label='temps de calcul de FFT') # tracé en échelle log
idx_norm = np.round(np.log10(list_N[-1]*np.log(list_N[-1]))) # normalisation de l'échelle pour Nlog(N)
plt.loglog(list_N,list_N*np.log(list_N)/10**idx_norm,'--k',label=r'$N \log(N)$')
plt.xlim(list_N.min(),list_N.max())
plt.legend(loc='best',fontsize=14)
plt.show()

On peut donc en conclure que ❓❓❓

## Vérification du théorème de Plancherel

Comme énoncé en préambule, le théorème de Plancherel stipule que $\mathcal{F}(x\ast y) = \mathcal{F}(x)\times \mathcal{F}(y)$, autrement dit $(x\ast y) = \mathcal{F}^{-1}\big(\mathcal{F}(x)\times \mathcal{F}(y)\big)$. Numériquement, cela permet de réduire la complexité effective du calcul de $(x\ast y)$ de $\mathcal{O}(N^2)$ par la formule directe à $\mathcal{O}(N \log(N))$ pour des signaux de longueur $N$ grâce à l'application de la FFT. On va donc vérifier en pratique cette relation.

Pour cela, il est important de noter que [`sp.signal.convolve`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve.html) permet de choisir la méthode de calcul de la convolution :
* `method='direct'` forcera le calcul de la convolution par la méthode directe, c'est-à-dire $
(x \ast y)[n] = \displaystyle \sum_{k=0}^{N-1} x[k] \cdot y[n-k] $ $\forall n \in [0,N-1]$. Cette formule montre bien que $N$ multiplications sont nécessaires pour calculer une valeur du produit de convolution. Ce dernier en possédant $N$, la complexité est bien en $O(N^2)$
* `method='fft'` calculera la convolution grâce à la FFT et au théorème de Plancherel (en appelant [`sp.signal.fftconvolve`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.fftconvolve.html#scipy.signal.fftconvolve) sous le manteau...). Le produit terme à terme des 2 FFTs $\mathcal{F}(x)\times \mathcal{F}(y)$ étant en $O(N)$, c'est donc le calcul des FFT directes $\mathcal{F}(\ )$ et inverse $\mathcal{F}^{-1}( \ )$ en $O (N \log N)$ qui domine la complexité générale.

À noter que par défaut, c'est en fait `method='auto'` qui est utilisé par `sp.signal.convolve`, qui détermine automatiquement quelle méthode de calcul est la plus rapide en fonction de la taille du signal, et applique de fait cette méthode.

Comme pour la question précédente, il va donc falloir définir une liste de tailles $N$ pour les signaux dont on va monitorer le temps de calcul de la convolution. Sauf que cette fois ci, on va comparer (à priori) du $O(N^2)$ avec du $O(N \log(N))$ : on va donc se restreindre à un ordre de grandeur d'au maximum $N \sim 2.10^5$, sinon ça risque de mouliner longtemps pour le $O(N^2)$... De plus, puisqu'on va de nouveau afficher les temps de calcul en échelle log, on va directement échantillonner de manière logarithmique (c'est-à-dire régulièrement vis-à-vis de l'échelle log) grâce à [`np.geomspace`](https://numpy.org/doc/stable/reference/generated/numpy.geomspace.html)

In [ ]:
# 🛑 liste de tailles de signaux (échantillonnage logarithmique)
list_N = np.round(np.geomspace(10,2e5,num=30)).astype(int)
print("Liste des valeurs régulières en échelle log :",list_N)

### 🛠️ 🚧 👷  À vous de jouer !

Complétez la cellule suivante pour calculer, pour chaque longueur $N$ dans `list_N`, le temps de calcul de la convolution par la méthode directe et par la FFT d'un signal de longueur $N$ (pour une forme de signal donnée). Tracez ensuite ces temps de calcul en fonction de $N$. <br>
La méthode directe est elle bien asymptotiquement en $O(N^2)$ et la méthode FFT en $O(N \log(N))$ ? L'évolution du temps de calcul est elle dépendante de la forme du signal choisi ?
Au final, à partir de quel ordre de grandeur pour $N$ est-ce plus intéressant d'utiliser la méthode FFT que la méthode directe pour le calcul de la convolution ?

In [ ]:
t_elapse_direct = [] # initialisation de la liste pour stocker les temps de calcul par la méthode directe
t_elapse_fft = [] # initialisation de la liste pour stocker les temps de calcul par la FFT

for N in list_N:
    x = generate_??? # FIXME génère un signal aléatoire/dent de scie/carré de longueur N
    
    # Temps de calcul de la convolution - méthode directe
    t_begin_direct = ??? # FIXME début chronomètre
    dummy_conv = ??? # FIXME calcul de la convolution par la méthode directe
    t_end_direct = ??? # FIXME fin chronomètre
    t_elapse_direct.append(t_end_direct - t_begin_direct) # temps de calcul par la méthode directe
    
    # Temps de calcul de la convolution - méthode FFT
    t_begin_fft = ??? # FIXME début chronomètre
    dummy_conv = ??? # FIXME calcul de la convolution par la méthode FFT
    t_end_fft = ??? # FIXME fin chronomètre
    t_elapse_fft.append(t_end_fft - t_begin_fft) # temps de calcul par la méthode FFT

In [ ]:
# 🛑 Comparaison des temps de calcul des deux méthodes
plt.figure(figsize=(8,5))
plt.loglog(list_N,t_elapse_direct,'b',label='Convolution - méthode directe') # tracé en échelle log
idx_norm_direct = np.round(np.log10(list_N[-1]**2/t_elapse_direct[-1])) # normalisation de l'échelle pour N^2
plt.loglog(list_N,list_N**2/10**idx_norm_direct,'--b',label=r'$N^2$')
plt.loglog(list_N,t_elapse_fft,'r',label='Convolution - méthode FFT') # tracé en échelle log
idx_norm_fft = np.round(np.log10(list_N[-1]*np.log(list_N[-1])/t_elapse_fft[-1])) # normalisation de l'échelle pour Nlog(N)
plt.loglog(list_N,list_N*np.log(list_N)/10**idx_norm_fft,'--r',label=r'$N\log(N)$')
plt.xlim(list_N.min(),list_N.max())
plt.legend(loc='best',fontsize=14)
plt.show()

On peut donc en conclure que ❓❓❓ <br>
L'ordre de grandeur à partir duquel la méthode FFT devient plus intéressante que la méthode directe du point de vue du temps de calcul est $N \sim $ ❓❓❓

# Bravo !
Vous en avez fini avec cet exercice sur le théorème de Plancherel et ses conséquences sur la complexité du calcul de la convolution ! Vous pouvez donc maintenant attaquer l'un des deux exercices suivant (si pas déjà fini) :
* [Traitement d'un signal d'ECG cardiaque](TP3_Fourier_application_ECG.ipynb)
* [Filtrage d'un signal grâce à la transformée de Fourier](TP3_Fourier_application_filtrage.ipynb)